<a href="https://colab.research.google.com/github/Zann181/Parcial-2_SyS/blob/main/Dashboards_Streamlit_Ngrok_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guía Streamlit en Colab y Ngrok - Dashboards

Elaborado por: Juan David Muñoz Buritica jmunozbu@unal.edu.co (Monitoria SyS 2024-1)
Revisado: Andrés Marino Álvarez Meza amalvarezme@unal.edu.co

- [Streamlit](https://streamlit.io/) es una librería destinada a crear y compartir aplicaciones o dashboards.
- Su objetivo es ser una herramienta fácil de usar que permita ejecutar scripts directamente y desplegarlos en un aplicativo web.
- Sin embargo, antes de ver su funcionamiento, debemos tener en cuenta que, si bien Streamlit permite correr el aplicativo, debemos alojarlo y "hacerlo visible" a la red.
- Existen diferentes métodos que cumplen dicho objetivo, también dependiendo del nivel de madurez que se requiera, bien sea simplemente testeo o producción. - Una alternativa eficaz, rápida y que puede ser fácilmente escalable es [Ngrok](https://ngrok.com/).

- Ngrok es un servicio y aplicativo globalmente distribuido que asegura, acelera y protege las aplicaciones y servicios de red.
- En este caso, nos servirá para crear el tunel http que nos deje visualizar el dashboard corriendo con Streamlit desde el entorno del cuadernillo o script de Python.
- El servicio gratuito de Ngrok, tras crear la cuenta tiene la posibilidad de mantener un agente activo simultáneo, cuya configuración está dada por el token de autenticación único por usuario y túnel.

**Nota: Cree una cuenta en Ngrok https://ngrok.com/ con su correo UNAL y guarde el token personal asignado**

In [1]:
!pip install streamlit -q #instalación de librerías
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.0 MB/s eta 0:00:00


In [2]:
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile #librerias descarga Youtube y manejo de audios en python

     - 2.7 MB 8.5 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3.8-py3-none-any.whl.metadata (9.9 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 45.1 MB/s eta 0:00:00
Using cached charset

In [3]:
# Instalar ffmpeg si es necesario en Colab
!apt-get install ffmpeg

import os
import streamlit as st
from yt_dlp import YoutubeDL

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


La estructura básica para correr un programa o script en streamlit es la siguiente.

- A continuación se muestra una app básica con algunas de las funcionalidades básicas para visualizar en dashboard.

- Debemos crear un archivo .py con los códigos del dashboard para su posterior visualización en Ngrok.

In [43]:
%%writefile detector.py

import streamlit as st
import numpy as np
import os
import soundfile as sf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from pyngrok import ngrok
import yt_dlp as youtube_dl
import joblib
import warnings
import scipy.signal as sig

warnings.filterwarnings("ignore")



# Clase principal para el procesamiento de señales
class SignalProcessor:
    def __init__(self, A=1, T=5, Fs=100, N=50):
        self.A = A
        self.T = T
        self.Fs = Fs
        self.N = N
        self.F_o = 1 / self.T
        self.tv = np.arange(-self.T / 2, self.T / 2, 1 / self.Fs)
        self.xo = self.A ** 2 * np.sin(2 * np.pi * self.F_o * self.tv) ** 2
        self.wo = 2 * np.pi / self.T
        self.phin = np.zeros((len(self.tv), 2 * self.N + 1), dtype=np.complex_)
        self.Cn = np.zeros(2 * self.N + 1, dtype=np.complex_)
        self.nv = np.linspace(-self.N, self.N, 2 * self.N + 1)
        self.Px = self.A ** 2
        self._initialize_signal_bases()
        self._initialize_coefficients()

    def _initialize_signal_bases(self):
        for n in range(-self.N, self.N + 1):
            self.phin[:, n + self.N] = np.exp(1j * n * self.wo * self.tv)

    def _initialize_coefficients(self):
        self.Cn[self.N] = self.A ** 2 / 2
        self.Cn[self.N + 2] = self.Cn[self.N - 2] = -self.A ** 2 / 4

    def plot_base(self, n):
        fig, ax = plt.subplots()
        ax.plot(self.tv, np.real(self.phin[:, int(n + self.N)]), label=r"Re$\phi_n$(t)")
        ax.plot(self.tv, np.imag(self.phin[:, int(n + self.N)]), label=r"Im$\phi_n$(t)")
        ax.plot(self.tv, self.xo, 'r', label="x(t)")
        ax.set_xlabel("Tiempo [s]")
        ax.set_ylabel("Base")
        ax.grid(True)
        ax.legend()
        st.pyplot(fig)

    def plot_spectrum(self):
        fig, axs = plt.subplots(2, 2, figsize=(8, 8))
        axs[0, 0].stem(self.nv, np.real(self.Cn))
        axs[0, 0].set_title(r"Re$(C_n)$")
        axs[0, 1].stem(self.nv, np.imag(self.Cn))
        axs[0, 1].set_title(r"Im$(C_n)$")
        axs[1, 0].stem(self.nv, np.abs(self.Cn))
        axs[1, 0].set_title(r"$|C_n|$")
        axs[1, 1].stem(self.nv, np.angle(self.Cn))
        axs[1, 1].set_title(r"$\angle C_n$")
        for ax in axs.flat:
            ax.grid(True)
        plt.tight_layout()
        st.pyplot(fig)

    def plot_reconstruction(self, Na):
        indices = np.arange(self.N - Na, self.N + Na + 1)
        error = 1 - np.sum(np.abs(self.Cn[indices]) ** 2) / self.Px
        estimated_signal = self.phin[:, indices].dot(self.Cn[indices])
        fig, ax = plt.subplots()
        ax.plot(self.tv, np.real(estimated_signal), label="x_e(t) Estimada")
        ax.plot(self.tv, self.xo, 'r', label="x(t) Original")
        ax.set_title(f"Error Relativo: {100 * error:.2f}%")
        ax.grid(True)
        ax.legend()
        st.pyplot(fig)



# Clase para descargar y procesar audio desde YouTube
class YouTubeAudioProcessor:
    @st.cache_data
    def download_ytvid_as_mp3(_self, video_url, name):
        video_info = youtube_dl.YoutubeDL().extract_info(url=video_url, download=False)
        filename = f"{name}.mp3"
        options = {
            'format': 'bestaudio/best',
            'keepvideo': False,
            'outtmpl': filename,
        }

        with youtube_dl.YoutubeDL(options) as ydl:
            ydl.download([video_info['webpage_url']])
        return filename

    def convert_mp3_to_wav(self, mp3_file, wav_file):
        os.system(f"ffmpeg -i {mp3_file} {wav_file}")

    def process_wav_file(self, nombre_out, ti, tf, fs, selection):
        xa, fs = sf.read(nombre_out)
        st.write(f'Frecuencia de muestreo: {fs:.2f} Hz\nArchivo de audio: {nombre_out}')

        # Extraer los 5 segundos entre ti y tf
        start_sample = int(ti * fs)
        end_sample = int(tf * fs)
        fragmento = xa[start_sample:end_sample]

        # Guardar el fragmento en un nuevo archivo temporal
        temp_filename = "fragmento.wav"
        sf.write(temp_filename, fragmento, fs)

        # Reproducir el fragmento de 5 segundos
        st.audio(temp_filename)

        if selection == 'Portadora':
            self.plot_carrier_signal(fragmento, fs, ti, tf)
        elif selection == 'Modulada':
            self.plot_modulated_signal(fragmento, fs, ti, tf)

    def plot_carrier_signal(self, m, fs, ti, tf):
        tt = np.arange(0, tf - ti, 1 / fs)
        Fc = 15000
        Im = 0.5
        Ac = np.max(abs(m)) / Im
        c = Ac * np.sin(2 * np.pi * Fc * tt)

        Cw = np.fft.rfft(c)
        vf = np.fft.rfftfreq(len(c), d=1/fs)

        fig, ax = plt.subplots()
        ax.plot(vf, abs(Cw))
        ax.set_title(r'Espectro Portadora')
        ax.set_xlabel(r'$f[Hz]$', fontsize=14)
        ax.set_ylabel(r'$|C[f]|$', fontsize=14)
        ax.grid(True)
        st.pyplot(fig)

    def plot_modulated_signal(self, m, fs, ti, tf):
        tt = np.arange(0, tf - ti, 1 / fs)
        Fc = 15000
        Im = 0.5
        Ac = np.max(np.abs(m[:, 0])) / Im  # Seleccionar el primer canal
        c = Ac * np.sin(2 * np.pi * Fc * tt)

        y = (1 + m[:, 0] / Ac) * c  # Usar el primer canal
        Yw = np.fft.rfft(y)
        vf = np.fft.rfftfreq(len(y), d=1/fs)

        fig, ax = plt.subplots()
        ax.plot(vf, np.abs(Yw))
        ax.set_title(r'Espectro Señal Modulada')
        ax.set_xlabel(r'$f[Hz]$', fontsize=14)
        ax.set_ylabel(r'$|Y[f]|$', fontsize=14)
        ax.grid(True)
        st.pyplot(fig)



# Clase para simular circuitos RC
class CircuitSimulator:
    def __init__(self):
        self.Fo = 60
        self.Fs = 30 * self.Fo
        self.To = 1 / self.Fo
        self.Ts = 1 / self.Fs
        self.t = np.arange(0, 5 * self.To, self.Ts)
        self.A = 120
        self.in_o = self.A * np.abs(np.sin(2 * np.pi * self.Fo * self.t))  # Onda senoidal rectificada

    def transferencia_RC(self, R, C):
        num = [1]
        den = [R * C, 1]
        return sig.TransferFunction(num, den)

    def simular_salida(self, R, C):
        G_n = self.transferencia_RC(R, C)
        _, out, _ = sig.lsim(G_n, self.in_o, self.t)
        return out

    def calcular_fft(self, signal):
        vfre = np.fft.rfftfreq(len(signal), self.Ts)
        Xf = np.fft.rfft(signal)
        return vfre, Xf

    def calcular_thd(self, Xf):
        V1 = np.abs(Xf[1]) / len(Xf)
        Vh = np.sqrt(np.sum(np.abs(Xf[2:]) ** 2)) / len(Xf)
        THD = Vh / V1
        return THD

    def simulacion(self, R_v, C_v):
        # Simulación de salida para la resistencia pura y el circuito RC usando las mismas variables
        out_res = self.simular_salida(R_v, 1e-12)  # Carga resistiva (capacitancia despreciable)
        out_RC = self.simular_salida(R_v, C_v)  # Circuito RC serie

        # Cálculo del espectro de ambas salidas
        vfre_res, Xf_res = self.calcular_fft(out_res)
        vfre_RC, Xf_RC = self.calcular_fft(out_RC)

        # Graficar espectro para el caso resistivo
        self.plot_espectro(vfre_res, Xf_res, "Resistiva")

        # Graficar espectro para el caso RC serie
        self.plot_espectro(vfre_RC, Xf_RC, "RC Serie")

        # Cálculo del THD
        THD_res = self.calcular_thd(Xf_res)
        THD_RC = self.calcular_thd(Xf_RC)

        st.write(f'THD (Carga Resistiva) = {THD_res * 100:.2f}%')
        st.write(f'THD (Carga RC Serie) = {THD_RC * 100:.2f}%')

    def plot_espectro(self, vfre, Xf, label):
        st.write(f"Espectro de Frecuencia - Carga {label}")
        fig, ax = plt.subplots()
        ax.stem(vfre, np.abs(Xf) / len(Xf), basefmt=" ", use_line_collection=True)
        ax.set_xlabel('Frecuencia (Hz)')
        ax.set_ylabel('Magnitud')
        ax.grid(True)
        st.pyplot(fig)


# Código principal de Streamlit
def main():
    st.title("Parcial#2")

    # Instancia de la clase SignalProcessor
    signal_processor = SignalProcessor()

    # Widget interactivo para seleccionar el armónico n
    n = st.slider('Elige el armónico n', min_value=-signal_processor.N, max_value=signal_processor.N, value=0, step=1)
    signal_processor.plot_base(n)

    # Gráfica del espectro de Fourier
    signal_processor.plot_spectrum()

    # Slider para seleccionar Na
    Na = st.slider('Elige el número de armónicos Na', min_value=1, max_value=signal_processor.N, value=1, step=1)
    signal_processor.plot_reconstruction(Na)

    # Sección de modulación AM
    st.write('Ingrese la URL de YouTube:')
    custom_url = st.text_input("URL de YouTube:", "https://www.youtube.com/watch?v=aZiQ7aJ_BVo")
    yt_audio = YouTubeAudioProcessor()

    selection = st.radio("Seleccione el tipo de señal para graficar:", ('Portadora', 'Modulada'))

    if st.button("Descargar y procesar audio"):
        mp3_filename = yt_audio.download_ytvid_as_mp3(custom_url, "audio")
        wav_filename = "output.wav"
        yt_audio.convert_mp3_to_wav(mp3_filename, wav_filename)
        yt_audio.process_wav_file(wav_filename, ti=20, tf=25, fs=44100, selection=selection)

    # Simulación de circuitos
    st.title("Simulación de Circuitos RC")

    # Entradas comunes de resistencia y capacitancia
    R_v = st.number_input("Resistencia (R) para el circuito:", min_value=1, value=1000)
    C_v = st.number_input("Capacitancia (C) para el circuito:", min_value=0.0, value=1e-6)

    if st.button("Ejecutar simulación"):
        simulator = CircuitSimulator()
        simulator.simulacion(R_v, C_v)


if __name__ == "__main__":
    main()


Overwriting detector.py


Como al momento de correr el script con Streamlit se requiere del archivo .py donde se encuentre el código, utilizamos %%writefile para crearlo. Posteriormente hacemos toda la configuración de Ngrok con el token y demás datos de protocolo.

In [5]:
token = '2loY5BE5ZxQ3TstzfOMcpiGVm5q_4wkwQQXBRgu5GmePE82kL' #colocar aquí su token personal después de crear su cuenta con correo UNAL en Ngrok

In [6]:
from pyngrok import ngrok

# Set authentication token (unique per user)
ngrok.set_auth_token(token)

# Start Streamlit server on a specific port
!nohup streamlit run detector.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://0d5e-34-82-37-207.ngrok-free.app
